# Metadata Replacement + Node Sentence Window

이 노트북에서는 SentenceWindowNodeParser를 사용하여 문서를 노드당 단일 문장으로 구문 분석합니다. 
각 노드에는 노드 문장의 양쪽에 문장이 있는 "창"도 포함되어 있습니다.


그런 다음 검색 중에 검색된 문장을 LLM에 전달하기 전에 단일 문장은 MetadataReplacementNodePostProcessor를 사용하여 주변 문장을 포함하는 창으로 대체됩니다.

이는 보다 세밀한 세부 정보를 검색하는 데 도움이 되므로 대규모 문서/인덱스에 가장 유용합니다.

기본적으로 문장 창은 원래 문장의 양쪽에 5개의 문장입니다. 이 경우 창 설정을 따르기 위해 청크 크기 설정이 사용되지 않습니다.

In [1]:
import logging
import sys
import os

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

from dotenv import load_dotenv
load_dotenv()

True

In [2]:
from llama_index.llms.openai import OpenAI
from llama_index.core import Settings

Settings.llm = OpenAI(model="gpt-4-1106-preview", temperature=0)

In [3]:
import json

with open('E-9_Visa_Docs.json', 'r') as f:
    visa_docs_json = json.load(f)

In [4]:
""" Load Documents from JSON file """

from llama_index.core.schema import Document

visa_docs = [
    Document(
        text=doc["text"], 
        metadata=doc["metadata"], 
        excluded_llm_metadata_keys=doc["excluded_llm_metadata_keys"], 
        excluded_embed_metadata_keys=doc["excluded_embed_metadata_keys"],
    ) 
    for doc in visa_docs_json
]

In [5]:
print(len(visa_docs))

21


In [ ]:
for doc in visa_docs:
    print(f"Page: {doc.metadata['page_label']}")
    print(f"{doc.text}\n\n")

In [6]:
from llama_index.core.node_parser import SentenceWindowNodeParser


node_parser = SentenceWindowNodeParser.from_defaults(
    window_size=3,
    window_metadata_key="window",
    original_text_metadata_key="original_text",
)


from llama_index.core import Settings
from llama_index.core.node_parser import SentenceSplitter

text_splitter = SentenceSplitter()
Settings.text_splitter = text_splitter

In [7]:
nodes = node_parser.get_nodes_from_documents(visa_docs)

In [8]:
len(nodes)

26

In [9]:
for node in nodes:
    print(f"Page: {node.metadata['page_label']}")
    print(f"{node.text}\n\n")

Page: 1

        # E-9: 고용허가제 비전문취업 비자

        고용허가제
            고용허가제 해당자와 활동범위
        고용허가제 해당자에 대한 사증 발급
            사증발급 후 1회 체류기간의 상한
            사증발급 허용업종 및 체류자격 약호(기호): E-9 근로자를 채용할 수 있는 업종과 범위
            사증발급인정서를 받아야만 E-9 사증을 발급받을 수 있음
            2012.8.1부터 범죄경력증명서 및 건강상태확인서 제출
            비전문취업(E-9) 자격 사증발급인정서 발급 절차
            재입국특례 제도(구 성실근로자 제도)
            재입국특례자에 대한 사증 신청 및 발급 방법
            재입국특례자에 대한 우대 내용
            고용허가제 해당자의 근무처(직장)의 변경
            고용허가제 해당자의 근무처(직장)를 변경할 수 있는 조건
            고용허가제 해당자의 근무처(직장)를 변경하는 절차 및 제출 서류
            고용허가제 농업 분야 외국인근로자의 근무처(직장) 추가
            고용허가제 해당자의 체류자격 변경허가 - 사증 변경
            고용허가제 해당자의 체류기간 연장 허가 - 사증 연장
            고용허가제 해당자의 재입국허가
            고용허가제 해당자의 외국인등록
            고용허가제 해당자의 고용변동 신고
        


Page: 2
## 고용허가제 
            - ｢외국인근로자의 고용 등에 관한 법률｣에 의거, 사업주에게 외국인근로자의 고용을 허가하고, 외국인 근로자에게는 당해 사업주에게 고용되는 조건으로 최장 4년 10개월간 취업을 허용하는 인력제도로, ‘04. 


Page: 2
8월 제도 시행 이후 현재까지 16개국과 MOU를 체결하여 운영
            - 상시근로자(고용보험

In [ ]:
from llama_index.core import VectorStoreIndex

sentence_index = VectorStoreIndex(nodes)

In [ ]:

from llama_index.core.postprocessor import MetadataReplacementPostProcessor

query_engine = sentence_index.as_query_engine(
    similarity_top_k=2,
    # the target key defaults to `window` to match the node_parser's default
    node_postprocessors=[
        MetadataReplacementPostProcessor(target_metadata_key="window")
    ],
)

In [ ]:
query = "저는 한국에 살고 있는 외국인인데, 직업을 바꾸는게 가능한가요?"
query = "한국에서 건물 청소하는 일을 할 예정인데 건물 청소는 어떤 업종이고 체류자격의 비자 타입은 무엇인가요?"
query = "현재 농장에서 일하고 있는데 이 분야가 계절적 특성을 타기 때문에 다음주 부턴 일이 없는데, 다른 분야의 사업장에 일을 구하는게 가능한가요?"
query = "올해부터 E9 비자로 식당에서 일을 할 수 있는 것 맞나요?"


window_response = query_engine.query(query)
print(window_response)

In [ ]:
window = window_response.source_nodes[0].node.metadata["window"]
sentence = window_response.source_nodes[0].node.metadata["original_text"]

print(f"Window: {window}")
print("------------------")
print(f"Original Sentence: {sentence}")